In [70]:
# Get rid of warning messages and CUDA loadings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'

# Import Packages
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io
import matplotlib.animation as animation
from matplotlib import cm
import time

In [71]:
Adam_Epochs = 100 # Number of epochs of Adam optimization
q = 100 # Number of RK time steps (max 500)
noise = 0.01 # number of standard deviations of output noise
D=1.001875195652246475e-02
r=5.011364221572875977e-01

In [76]:
#load data
path='data/Fisher_3D_Symmetric.mat'
data=scipy.io.loadmat(path)
t = data['t'].flatten()[:,None]
x = data['x'][:,0].flatten()[:,None]
y = data['x'][:,1].flatten()[:,None]
z = data['x'][:,2].flatten()[:,None]
Exact_u = np.abs(data['u'])
#create a time/space grid
X, T = np.meshgrid(x,t)
Y, T = np.meshgrid(y,t)
Z, T = np.meshgrid(z,t)
#flatten X and T
xt = np.hstack((X.flatten()[:,None], T.flatten()[:,None])) #flatten X and T
yt = np.hstack((Y.flatten()[:,None], T.flatten()[:,None])) #flatten Y and T
zt = np.hstack((Z.flatten()[:,None], T.flatten()[:,None])) #flatten Z and T
xyzt = np.array([xt[:,0], yt[:,0], zt[:,0], xt[:,1]]).T # all (x,y,t) combos in an array (shape = (x*y*z*t, 4))

In [86]:
x.shape

(29791, 1)

In [77]:
u_star = np.array([])
for i in range(len(t)):
    u_star = np.append(u_star, Exact_u[:,:,:,i].T.flatten())
u_star = u_star.reshape(len(u_star),1) # values of the solution corresponding to the inputs of xyt
u_star.shape,xyzt.shape #31*31*31*101，data['x'].shape

((3008891, 1), (3008891, 4))

In [97]:
#define the model structure
Inputs=tf.keras.Input(shape=(3,))#1D input
layer_number=100
Dense_1=tf.keras.layers.Dense(layer_number,activation='tanh',kernel_initializer="glorot_normal")(Inputs)
Dense_2=tf.keras.layers.Dense(layer_number,activation='tanh',kernel_initializer="glorot_normal")(Dense_1)
Dense_3=tf.keras.layers.Dense(layer_number,activation='tanh',kernel_initializer="glorot_normal")(Dense_2)
Dense_4=tf.keras.layers.Dense(layer_number,activation='tanh',kernel_initializer="glorot_normal")(Dense_3)
#output layer dimension is q
Predictions=tf.keras.layers.Dense(q)(Dense_4)
model_predict=tf.keras.Model(inputs=Inputs,outputs=Predictions)

In [98]:
#parameter for initial condition
IRK_weights=np.float32(np.loadtxt('data/Butcher_IRK100.txt',ndmin=2))
IRK_times = IRK_weights[q**2+q:]
IRK_weights = IRK_weights[:q**2+q].reshape((q+1,q))
IRK_alpha = tf.constant(IRK_weights[:-1,:], dtype='float32')
IRK_beta = tf.constant(IRK_weights[-1:,:], dtype='float32')

In [99]:
inputs = xyzt[:int(np.cbrt(x.shape[0]))*int(np.cbrt(y.shape[0]))*int(np.cbrt(z.shape[0])),0:3]
outputs_t0 = u_star[:int(np.cbrt(x.shape[0]))*int(np.cbrt(y.shape[0]))*int(np.cbrt(z.shape[0])),0] # Initial snapshot solution data
outputs_t1 = u_star[-int(np.cbrt(x.shape[0]))*int(np.cbrt(y.shape[0]))*int(np.cbrt(z.shape[0])):,0] # Final snapshot solution data
outputs_t0_noise = outputs_t0 + noise*np.std(outputs_t0)*np.random.randn(outputs_t0.shape[0]) # Initial snapshot solution data with noise
outputs_t1_noise = outputs_t1 + noise*np.std(outputs_t1)*np.random.randn(outputs_t1.shape[0]) # Final snapshot solution data with noise
dt=(t[-1]-t[0])[0]
#normalize the data
input_min=inputs.min(0)#minimum of each column
input_max=inputs.max(0)#maximum of each column
inputs_norm=2*(inputs-input_min)/(input_max-input_min)-1.0#normalize the inputs

#define tf,variables for neural network
inputs_var = tf.Variable(inputs_norm, name='inputs_var')
dummy=tf.ones((inputs_var.shape[0],q),dtype=np.float32)

In [100]:
xyzt.shape

(3008891, 4)

In [101]:
##define solution of the PDE
class PDEsolution(tf.keras.layers.Layer):
    def __init__(self, D_var, r_var):
        super(PDEsolution, self).__init__()
        self.D_var = tf.Variable(np.array([D_var]))
        self.r_var = tf.Variable(np.array([r_var]))

    def loss(self,pred):   
        with tf.GradientTape(persistent=True) as tape:
                tape.watch(inputs_var)
                tape.watch(dummy)
                k = model_predict(inputs_var)
                g_U = tape.gradient(k, inputs_var, output_gradients=dummy) #U 关于x和y的导数
                g_Ux = g_U[:,0] #U 关于x的导数
                g_Uy = g_U[:,1]  #U 关于y的导数
                g_Uz = g_U[:,2]  #U关于z的导数 
                k_x = tape.gradient(g_Ux, dummy)
                k_y = tape.gradient(g_Uy, dummy)
                k_z = tape.gradient(g_Uz, dummy)
                g_Ux_U=tape.gradient(k_x,inputs_var,output_gradients=dummy) #Ux 关于x和y的导数
                g_Uy_U=tape.gradient(k_y,inputs_var,output_gradients=dummy) #Uy 关于x和y的导数
                g_Uz_U=tape.gradient(k_z,inputs_var,output_gradients=dummy) #Uz 关于x和y的导数
                g_UxUx = g_Ux_U[:,0] #Ux 关于x的导数
                g_UyUy = g_Uy_U[:,1] #Uy 关于y的导数
                g_UzUz = g_Uz_U[:,2] #Uz 关于z的导数
                k_xx = tape.gradient(g_UxUx, dummy) #U 关于x的二阶导数
                k_yy = tape.gradient(g_UyUy, dummy) #U 关于y的二阶导数
                k_zz = tape.gradient(g_UzUz, dummy) #U 关于y的二阶导数
        scale_x = 2/(input_max[0]-input_min[0]) # Define scale factors and scale derivatives
        scale_y = 2/(input_max[1]-input_min[1]) # Define scale factors and scale derivatives,因为前面进行了归一化
        scale_z = 2/(input_max[2]-input_min[2])
        k_x = k_x*scale_x
        k_y = k_y*scale_y
        k_z = k_z*scale_z
        k_xx = k_xx*scale_x**2
        k_yy = k_yy*scale_y**2
        k_zz = k_zz*scale_z**2
    
    
        #implement the PDE
        D = tf.cast(tf.exp(self.D_var), tf.float32)
        r = tf.cast(tf.exp(self.r_var), tf.float32)
        u = k
        u_x = k_x
        u_y = k_y
        u_z = k_z
        u_xx = k_xx
        u_yy = k_yy
        u_zz = k_zz
        #change it to float32
        u_xx = tf.cast(u_xx, tf.float32)
        u_yy = tf.cast(u_yy, tf.float32)
        u_zz = tf.cast(u_zz, tf.float32)
        u= tf.cast(u, tf.float32)
        #u_t is float32 do not match the u_xx
        u_t = tf.cast(D*(u_xx+u_yy+u_zz) + r*u*(1.0-u) , tf.float32)

        # condition for the initial condition
        dt = (t[-1] - t[0])[0] #total time
        U0_pred=k-dt*tf.matmul(u_t,tf.transpose(IRK_alpha))
        U1_pred=k+dt*tf.matmul(u_t,tf.transpose(IRK_beta-IRK_alpha))
        # Format exact solutions for loss
        U0 = q*[outputs_t0_noise]
        U0 = tf.stack(U0, axis=1)   
        U0 = tf.cast(U0, tf.float32)
        U1 = q*[outputs_t1_noise]
        U1 = tf.stack(U1, axis=1)
        U1 = tf.cast(U1, tf.float32)

        # Calculate loss
        MSE = tf.reduce_mean(tf.square(U0 - U0_pred)) + tf.reduce_mean(tf.square(U1 - U1_pred))
        return MSE

    
    def call(self,pred):              #call the loss function
        self.add_loss(self.loss(pred))    #add loss to the model
        return pred       

In [102]:
D_guess = np.log(D)
r_guess = np.log(r)
My_loss=PDEsolution(D_guess,r_guess)(Predictions)
# Create trainable model with custom loss function

In [ ]:
model_loss = tf.keras.Model(inputs=Inputs, outputs=My_loss)
model_loss.compile(optimizer=tf.keras.optimizers.Adam())
# Execute Adam optimization
inputs_var=tf.convert_to_tensor(inputs_var)
history=model_loss.fit(inputs_var,None, epochs=Adam_Epochs,verbose=0)  #None means no label